In [1]:
### Needed for google colab runtime
# import sys
# !{sys.executable} -m pip install pandas_datareader yfinance yahoo_fin pandas numpy openpyxl --upgrade --no-cache-dir

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import os, os.path
import errno
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
import asyncio
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import numpy as np
import multiprocessing
import datetime
import time
yf.pdr_override()

In [ ]:
#create directory if non-existent
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise
mkdir_p('us_stocks')

In [3]:
### Variables
tickers = pd.DataFrame()
#stock list downloaded from https://stockmarketmba.com/stocksintherussell1000.php
tickers = pd.read_excel("Russell1000list.xlsx",sheet_name=0,header=None,usecols = "A,E",skiprows = 2,names = ["ticker","sector"])
tickers["ticker"] = [item.replace(".", "-") for item in tickers["ticker"]] # Yahoo Finance uses dashes instead of dots
tickers = tickers.set_index("sector")

#combine into one string needed to download multiple files using yfinance module
tickers_string = ""
for i in tickers["ticker"]:
    tickers_string += i + " "

index_name = '^RUI' # Russell 1000
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

In [4]:
###===== OPTIONAL: if the file screener_yyfinance.ipynb has already been run =====###

###  Download ticker data from yahoo finance simultaneously
# results = yf.download(tickers = tickers_string,start = start_date, end = end_date, interval= '1d',threads = True,group_by="ticker")

### write to file
# for ticker in tickers["ticker"]:
    # df = results[ticker].reset_index()
    # df.to_csv( os.path.join(os.getcwd(),'us_stocks',ticker+'.csv'), index = False, header=True )

In [5]:
# Index Returns
index_df = pdr.get_data_yahoo(index_name, start_date, end_date)
index_return = index_df['Adj Close'].iloc[-1] / index_df['Adj Close'].iloc[0]       # not efficient: (index_df['Percent Change'] + 1).cumprod()[-1]

# Import historical data as CSV for all stocks (makes the process faster)
df_list = pd.concat(( map(pd.read_csv, [os.path.join(os.getcwd(),'us_stocks',s+'.csv') for s in tickers["ticker"] ])), keys=tickers["ticker"] )

# Find top 30% performing stocks (relative to the index)
returns_multiples = []
for ticker in tickers["ticker"]:
    # Calculating returns relative to the market (returns multiple)
    df = df_list.loc[ticker].copy()
    stock_return = df['Adj Close'].iloc[-1] / df['Adj Close'].iloc[0]   # not efficient: ((df['Percent Change'] + 1).cumprod()).iloc[-1]
    
    returns_multiple = round((stock_return / index_return), 2)
    returns_multiples.extend([returns_multiple])
    
    print (f'Ticker: {ticker}; Returns Multiple against index: {returns_multiple}\n')

[*********************100%***********************]  1 of 1 completed
Ticker: AAPL; Returns Multiple against index: 1.19

Ticker: MSFT; Returns Multiple against index: 1.01

Ticker: AMZN; Returns Multiple against index: 0.86

Ticker: TSLA; Returns Multiple against index: 1.36

Ticker: BRK-B; Returns Multiple against index: 1.15

Ticker: FB; Returns Multiple against index: nan

Ticker: UNH; Returns Multiple against index: 1.47

Ticker: META; Returns Multiple against index: 0.5

Ticker: NVDA; Returns Multiple against index: 0.72

Ticker: V; Returns Multiple against index: 1.03

Ticker: JNJ; Returns Multiple against index: 1.14

Ticker: XOM; Returns Multiple against index: 2.09

Ticker: PG; Returns Multiple against index: 1.11

Ticker: WMT; Returns Multiple against index: 1.06

Ticker: MA; Returns Multiple against index: 1.09

Ticker: JPM; Returns Multiple against index: 0.87

Ticker: HD; Returns Multiple against index: 1.03

Ticker: CVX; Returns Multiple against index: 1.93

Ticker: LLY; 

In [6]:
# Creating dataframe of only bottom 40%
tickers["Returns_multiple"] =  returns_multiples
tickers["RS_Rating"] = tickers.groupby("sector").Returns_multiple.rank(pct=True) * 100
rs_df = tickers[tickers.RS_Rating <= tickers.RS_Rating.quantile(.40)].sort_values('RS_Rating',ascending=True)

In [7]:
# Checking Minervini conditions of bottom 40% of stocks in given list
exportList = pd.DataFrame(columns=['Stock', "Sector", "RS_Rating", "returns multiple", "Last Price","%change","50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High", "% from 52 week low"])

for i in range(len(rs_df)):
    stock = rs_df["ticker"].iloc[i]
    ret_mult = rs_df["Returns_multiple"].iloc[i]
    sector = rs_df.index[i]
    try:
        df = pd.read_csv(os.path.join(os.getcwd(),'us_stocks',f'{stock}.csv'), index_col=0) 
        sma = [50, 150, 200]
        for x in sma:
            df["SMA_"+str(x)] = round(df['Adj Close'].rolling(window=x).mean(), 2)
        
        # Storing required values 
        currentClose = df["Adj Close"][-1]
        prevClose =  df["Adj Close"][-2]
        pct_chg   = (currentClose - prevClose ) * 100/ prevClose
        moving_average_50 = df["SMA_50"][-1]
        moving_average_150 = df["SMA_150"][-1]
        moving_average_200 = df["SMA_200"][-1]
        low_of_52week = round(min(df["Low"]), 2)
        high_of_52week = round(max(df["High"]), 2)
        pct_from_52wkhigh = (high_of_52week - currentClose)/high_of_52week * 100
        pct_from_52wklow = (currentClose - low_of_52week)/low_of_52week * 100
        RS_Rating = round(rs_df[rs_df['ticker']==stock].RS_Rating.tolist()[0])
        
        try:
            moving_average_200_20 = df["SMA_200"][-20]
        except Exception:
            moving_average_200_20 = 0

        # Condition 1: Current Price < 150 SMA and < 200 SMA
        condition_1 = currentClose < moving_average_150 < moving_average_200
        
        # Condition 2: 150 SMA and < 200 SMA
        condition_2 = moving_average_150 < moving_average_200

        # Condition 3: 200 SMA trending down for at least 1 month
        condition_3 = moving_average_200 < moving_average_200_20
        
        # Condition 4: 50 SMA< 150 SMA and 50 SMA< 200 SMA
        condition_4 = moving_average_50 < moving_average_150 < moving_average_200
           
        # Condition 5: Current Price < 50 SMA
        condition_5 = currentClose < moving_average_50
           
        # Condition 6: Current Price is at least 30% below 52 week high
        condition_6 = currentClose <= (0.7*high_of_52week)
           
        # Condition 7: Current Price is within 30% of 52 week low
        condition_7 = currentClose <= (1.4*low_of_52week)

        # Condition 8: Price greater than 15
        condition_8 = currentClose  > 15
        
        # If all conditions above are true, add stock to exportList
        if(condition_1 and condition_2 and condition_3 and condition_4 and condition_5 and condition_6 and condition_7 and condition_8):
            exportEntry = pd.DataFrame({'Stock': stock, "Sector" : sector, "RS_Rating": RS_Rating , "returns multiple" : ret_mult, "Last Price" : round(currentClose,3), "%change" : round(pct_chg,2), "50 Day MA": moving_average_50, "150 Day Ma" : moving_average_150, "200 Day MA": moving_average_200, "52 Week Low": low_of_52week, "52 week High": high_of_52week, "% from 52 week low" : round(pct_from_52wklow,2)},index=[0])
            exportList = pd.concat([exportList,exportEntry], ignore_index=True)
            print (stock + " made the Minervini requirements")
    except Exception as e:
        print (e)
        print(f"Could not gather data on {stock}")

exportList = exportList.sort_values(by='RS_Rating', ascending=True)
print('\n', exportList)
writer = ExcelWriter(os.path.join(os.getcwd(),"output_screener_us_short_position.xlsx"))
exportList.to_excel(writer, "Sheet1",index=False)
writer.save()

RNG made the Minervini requirements
UPST made the Minervini requirements
NVAX made the Minervini requirements
TREX made the Minervini requirements
TWLO made the Minervini requirements
TXG made the Minervini requirements
FRPT made the Minervini requirements
SWK made the Minervini requirements
W made the Minervini requirements
DOCU made the Minervini requirements
TDOC made the Minervini requirements
OKTA made the Minervini requirements
SMG made the Minervini requirements
SAM made the Minervini requirements
GNRC made the Minervini requirements
ROKU made the Minervini requirements
DASH made the Minervini requirements
ZM made the Minervini requirements
DOCS made the Minervini requirements
YETI made the Minervini requirements
ALGN made the Minervini requirements
BALL made the Minervini requirements
SPB made the Minervini requirements
MTCH made the Minervini requirements
EXAS made the Minervini requirements
PEGA made the Minervini requirements
PACW made the Minervini requirements
SITE made th

In [8]:
exportList

,Stock,Sector,RS_Rating,returns multiple,Last Price,%change,50 Day MA,150 Day Ma,200 Day MA,52 Week Low,52 week High,% from 52 week low
0,RNG,Information Technology,1,0.23,45.62,4.42,49.63,81.95,107.95,40.59,315.00,12.39
1,UPST,Financials,1,0.11,27.53,2.88,28.11,64.55,83.71,22.42,401.49,22.79
2,NVAX,Health Care,1,0.15,31.72,4.76,48.90,58.23,79.82,28.25,260.69,12.28
3,TREX,Industrials,1,0.50,49.73,3.11,55.75,63.90,78.05,44.49,140.98,11.78
4,TWLO,Information Technology,2,0.24,72.19,3.99,82.17,115.96,147.44,64.29,373.00,12.29
...,...,...,...,...,...,...,...,...,...,...,...,...
120,IART,Health Care,38,0.79,49.59,2.35,53.18,59.24,60.93,46.46,74.98,6.74
121,UHS,Health Care,38,0.79,103.01,1.75,105.60,123.09,124.31,94.70,158.28,8.78
124,TOL,Consumer Discretionary,38,0.84,45.34,2.53,47.36,48.19,52.55,40.20,75.61,12.79
125,DDOG,Information Technology,39,0.82,100.54,2.71,103.14,118.19,128.27,81.12,199.68,23.94


In [9]:
1+1

2